<H1>Interactive Visual Graphics with Folium</h1>

The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


In [1]:
%pip install piplite

ERROR: Could not find a version that satisfies the requirement piplite (from versions: none)

[notice] A new release of pip is available: 25.0.1 -> 26.0
[notice] To update, run: /Users/rhys/ml-env/bin/python -m pip install --upgrade pip
ERROR: No matching distribution found for piplite
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install folium


[notice] A new release of pip is available: 25.0.1 -> 26.0
[notice] To update, run: /Users/rhys/ml-env/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install pandas


[notice] A new release of pip is available: 25.0.1 -> 26.0
[notice] To update, run: /Users/rhys/ml-env/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import folium
import pandas as pd

In [5]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates

The following dataset with the name spacex_launch_geo.csv is an augmented dataset with latitude and longitude added for each site.

In [6]:
    import requests
    import io
    import pandas as pd
    
    URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
    resp = requests.get(URL)
    text = io.StringIO(resp.text)
    
    df = pd.read_csv(text)
    print('Data downloaded and read into a dataframe!')

Data downloaded and read into a dataframe!


Now, you can take a look at what are the coordinates for each site.


In [7]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.

We first need to create a folium Map object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.

In [8]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [9]:
site_map

We could use folium.Circle to add a highlighted circle area with a text label on a specific coordinate. For example,

In [10]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='blue', fill=True, opacity=0.2).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 20; color:"red";"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)
site_map

In [11]:
# Initialize the Feature Group
launch_locations = folium.map.FeatureGroup()

# Add circles and labels
    
for lat, lng, label in zip(launch_sites_df['Lat'], launch_sites_df['Long'], launch_sites_df['Launch Site']):
    
    # 1. Add the CircleMarker
    launch_locations.add_child(
        folium.CircleMarker(
            [lat, lng],
            radius=10,
            color='#d35400',
            fill=True,
            fill_color='#highlight',
            fill_opacity=0.6
        )
    )
    
    # 2. Add the Permanent Label using DivIcon
    launch_locations.add_child(
        folium.Marker(
            [lat, lng],
            icon=DivIcon(
                icon_size=(150,36),
                icon_anchor=(0,0), # Adjusts the position of the text relative to the coordinate
                html='<div style="font-size: 12pt; color: #d35400;"><b>%s</b></div>' % label,
            )
        )
    )

site_map.add_child(launch_locations)

Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.

# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not

In [12]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.

Let's first create a MarkerCluster object

In [13]:
marker_cluster = MarkerCluster()

TODO: Create a new column in spacex_df dataframe called marker_color to store the marker colors based on the class value

In [14]:
header_list = ["Launch Site", "Lat", "Long", "class", "marker_color"]
spacex_df_colours = spacex_df.reindex(columns = header_list)     

In [15]:
spacex_df_colours.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,NaN
47,KSC LC-39A,28.573255,-80.646895,1,NaN
48,KSC LC-39A,28.573255,-80.646895,1,NaN
49,CCAFS SLC-40,28.563197,-80.576820,1,NaN
50,CCAFS SLC-40,28.563197,-80.576820,1,NaN
51,CCAFS SLC-40,28.563197,-80.576820,0,NaN
52,CCAFS SLC-40,28.563197,-80.576820,0,NaN
53,CCAFS SLC-40,28.563197,-80.576820,0,NaN
54,CCAFS SLC-40,28.563197,-80.576820,1,NaN
55,CCAFS SLC-40,28.563197,-80.576820,0,NaN


In [16]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

def assign_marker_color(launch_outcome):
    # If the class is 1 (success), return green
    if launch_outcome == 1:
        return 'green'
    # Otherwise (failure), return red
    else:
        return 'red'

# 2. Apply the rule to the 'class' column and save it to 'marker_color'
spacex_df_colours['marker_color'] = spacex_df_colours['class'].apply(assign_marker_color)

In [17]:
spacex_df_colours.tail(20)

,Launch Site,Lat,Long,class,marker_color
36,KSC LC-39A,28.573255,-80.646895,1,green
37,KSC LC-39A,28.573255,-80.646895,0,red
38,KSC LC-39A,28.573255,-80.646895,1,green
39,KSC LC-39A,28.573255,-80.646895,1,green
40,KSC LC-39A,28.573255,-80.646895,0,red
41,KSC LC-39A,28.573255,-80.646895,1,green
42,KSC LC-39A,28.573255,-80.646895,1,green
43,KSC LC-39A,28.573255,-80.646895,0,red
44,KSC LC-39A,28.573255,-80.646895,1,green
45,KSC LC-39A,28.573255,-80.646895,1,green


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [18]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, row in spacex_df_colours.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    marker = folium.Marker(
        location=[row['Lat'], row['Long']],
        icon=folium.Icon(color='white', icon_color=row["marker_color"]))
    marker_cluster.add_child(marker)

site_map

In [19]:
%whos

Variable              Type             Data/Info
------------------------------------------------
DivIcon               type             <class 'folium.features.DivIcon'>
MarkerCluster         type             <class 'folium.plugins.ma<...>r_cluster.MarkerCluster'>
MousePosition         type             <class 'folium.plugins.mo<...>_position.MousePosition'>
URL                   str              https://cf-courses-data.s<...>ets/spacex_launch_geo.csv
assign_marker_color   function         <function assign_marker_color at 0x117b0b6a0>
circle                Circle           <folium.vector_layers.Cir<...>le object at 0x117b1d910>
df                    DataFrame        Shape: (56, 13)
folium                module           <module 'folium' from '/U<...>ages/folium/__init__.py'>
header_list           list             n=5
index                 int              55
io                    module           <module 'io' from '/Libra<...>12/lib/python3.12/io.py'>
label                 str         

# TASK 3: Calculate the distances between a launch site to its proximities


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [20]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [21]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

TODO: Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.

In [22]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163

launch_site_lat = 34.632834
launch_site_lon = -120.610745
coastline_lat = 34.63529
coastline_lon = -120.62473
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

In [23]:
distance_coastline

1.3087326952717047

In [24]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
distance_marker = folium.Marker(
    [34.63529, -120.62473],
    icon=DivIcon(
    icon_size=(20,20),
    icon_anchor=(0,0),
    html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
        )
   )
site_map.add_child(distance_marker)
site_map

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [29]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
coordinates = [[coastline_lat, coastline_lon], [launch_site_lat, launch_site_lon]]
lines=folium.PolyLine(locations=coordinates, weight = 1)
site_map.add_child(lines)

In [32]:
road_lat = 34.67212
road_lon = -120.56414
road_coordinates_line = [[road_lat, road_lon], [launch_site_lat, launch_site_lon]]

road_lines=folium.PolyLine(locations=road_coordinates_line, weight = 1)
site_map.add_child(road_lines)

In [35]:
distance_road = calculate_distance(launch_site_lat, launch_site_lon, road_lat, road_lon)

distance_marker2 = folium.Marker(
    [road_lat, road_lon],
    icon=DivIcon(
    icon_size=(20,20),
    icon_anchor=(0,0),
    html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_road),
        )
   )
site_map.add_child(distance_marker2)
site_map